In [53]:
import sqlite3
import os
from deltalake import DeltaTable
from deltalake.writer import write_deltalake
import numpy as np
import pandas as pd
conn = sqlite3.connect("database.sqlite")


In [2]:
user_df = pd.read_sql_query("select * from user;", conn)
grade_df = pd.read_sql_query("select * from grade;", conn)
method_df = pd.read_sql_query("select * from method;", conn)
ascent_df = pd.read_sql_query("select * from ascent;", conn)

In [56]:
# Write bronze data
write_deltalake("data/bronze/user", user_df, mode="overwrite")
write_deltalake("data/bronze/grade", grade_df, mode="overwrite")
write_deltalake("data/bronze/method", method_df, mode="overwrite")
write_deltalake("data/bronze/ascent", ascent_df, mode="overwrite")

In [4]:
# Read bronze data
fact_user_bronze_df = DeltaTable("data/bronze/user").to_pandas()
fact_ascent_bronze_df = DeltaTable("data/bronze/ascent").to_pandas()
dim_grade_bronze_df = DeltaTable("data/bronze/grade").to_pandas()
dim_method_bronze_df = DeltaTable("data/bronze/method").to_pandas()


In [5]:
# Apply transformations for silver table

In [6]:
# METHOD rename column name to climb_type
dim_method_silver_df = dim_method_bronze_df.rename(columns={'shorthand':'climb_type'})

In [7]:
# GRADE
dim_grade_columns = ['id', 'score', 'fra_routes', 'fra_boulders']
dim_grade_silver_df = dim_grade_bronze_df[dim_grade_columns]

In [8]:
# USER
fact_user_silver_df = fact_user_bronze_df[fact_user_bronze_df['deactivated'] == 0]
user_silver_columns = ['id', 'country', 'sex', 'height', 'weight', 'started']
fact_user_silver_df = fact_user_silver_df[user_silver_columns]

In [9]:
# ASCENT Add column to convert unix time to timestamp, select relevant columns and filter to year between 1980 and 2017
fact_ascent_bronze_df['datetime'] = pd.to_datetime(fact_ascent_bronze_df['date'], unit='s')
ascent_silver_columns = ['id','user_id', 'grade_id', 'method_id', 'climb_type', 'year', 'chipped']
fact_ascent_silver_df = fact_ascent_bronze_df[ascent_silver_columns]
fact_ascent_silver_df = fact_ascent_silver_df[(fact_ascent_silver_df['year'] >= 1980) & (fact_ascent_silver_df['year'] <= 2017)]

# Join to grade and method tables
fact_ascent_silver_grade_df = pd.merge(fact_ascent_silver_df, dim_grade_silver_df, left_on='grade_id', right_on='id',suffixes=('_ascent', '_grade'))
fact_ascent_silver_final_df = pd.merge(fact_ascent_silver_grade_df, dim_method_silver_df, left_on='method_id', right_on='id', suffixes=('_ascent', '_method'))
fact_ascent_silver_final_columns = ['id_ascent', 'user_id', 'year', 'chipped', 'score_ascent', 'fra_routes', 'fra_boulders', 'climb_type_method']
fact_ascent_silver_final_df = fact_ascent_silver_final_df.dropna(subset=['fra_routes'])
fact_ascent_silver_final_df = fact_ascent_silver_final_df[fact_ascent_silver_final_columns]

In [10]:
# Write silver tables to delta lake
write_deltalake("data/silver/user", fact_user_silver_df, mode="overwrite")
write_deltalake("data/silver/grade", dim_grade_silver_df, mode="overwrite")
write_deltalake("data/silver/method", dim_method_silver_df, mode="overwrite")
write_deltalake("data/silver/ascent", fact_ascent_silver_final_df, mode="overwrite")

In [12]:
# Read silver tables
fact_user_silver_df_2 = DeltaTable("data/silver/user").to_pandas()
fact_ascent_silver_df_2 = DeltaTable("data/silver/ascent").to_pandas()


In [45]:
# Create gold tables

# Average height by grade
merged_df = fact_user_silver_df_2.merge(fact_ascent_silver_df_2, left_on='id', right_on='user_id')
avg_height_by_grade_df = merged_df.groupby(['fra_routes'])['height'].mean().reset_index()

# Users grouped y countries
users_grouped_by_country_df = fact_user_silver_df_2['country'].value_counts().reset_index()

# Statistics of ascents and users by year
ascent_by_year_df = fact_ascent_silver_df_2[['user_id', 'year']]
ascent_by_year_df = ascent_by_year_df.groupby('year').agg(
    num_users=('user_id', 'nunique'), 
    num_ascents=('user_id', 'size'),
    median_ascents_per_user=('user_id', lambda x:x.value_counts().median())
).reset_index()

ascent_by_year_df['avg_ascents_per_user'] = ascent_by_year_df['num_ascents'] / ascent_by_year_df['num_users']

In [47]:
# Write gold tables to delta lake
write_deltalake("data/gold/avg_height_by_grade_year", avg_height_by_grade_df, mode="overwrite")
write_deltalake("data/gold/users_grouped_by_country", users_grouped_by_country_df, mode="overwrite")
write_deltalake("data/gold/ascents_by_year", ascent_by_year_df, mode="overwrite")

In [44]:
ascent_by_year

,year,num_users,num_ascents,median_ascents,avg_ascent_per_user
0,1980,9,33,2.0,3.666667
1,1981,4,46,4.0,11.500000
2,1982,8,82,1.5,10.250000
3,1983,9,78,1.0,8.666667
4,1984,16,54,1.5,3.375000
5,1985,20,110,3.0,5.500000
6,1986,33,226,3.0,6.848485
7,1987,43,300,3.0,6.976744
8,1988,40,375,2.5,9.375000
9,1989,66,577,4.0,8.742424


In [28]:
ascent_by_year = fact_ascent_silver_df_2[['user_id', 'year']]

In [29]:
ascent_by_year

,user_id,year
0,1,1999
1,1,1999
2,1,1999
3,1,1999
4,1,1999
...,...,...
4111773,59627,2017
4111774,44075,2017
4111775,19098,2017
4111776,19098,2017
